In [2]:
from pandas.io.json import json_normalize
import pandas as pd
import json
import numpy as np
import pandas.io.json as pd_json
from sqlalchemy import create_engine


In [3]:
jsonfile = pd.read_json("Resources/mergedjson.json")
jsonfile

modifiedjson = jsonfile.drop(['target_fg','target_og','contributed_by','boil_volume', 'method', 'volume'], axis = 1)

In [4]:
# getting rid of the NaNs in the IBU and ABV columns with column means
modifiedjson=modifiedjson.fillna(modifiedjson.mean())

In [5]:
#function to scale srm values based on scale
#https://www.twobeerdudes.com/beer/srm
def srmCat(s):
    if s['srm'] >= 1 and s['srm'] < 3:
        return 'Pale Straw'
    elif s['srm'] >=3 and s['srm'] < 4:
        return 'Straw'
    elif s['srm'] >=4 and s['srm'] < 6:
        return 'Pale Gold'
    elif s['srm'] >=6 and s['srm'] < 8:
        return 'Deep Gold'
    elif s['srm'] >=8 and s['srm'] < 10:
        return 'Pale Amber'
    elif s['srm'] >=10 and s['srm'] < 13:
        return 'Medium Amber'
    elif s['srm'] >=13 and s['srm'] < 17:
        return 'Deep Amber'
    elif s['srm'] >=17 and s['srm'] < 20:
        return 'Amber Brown'
    elif s['srm'] >=20 and s['srm'] < 24:
        return 'Brown'
    elif s['srm'] >=24 and s['srm'] < 35:
        return 'Ruby Brown'
    elif s['srm'] >=35 and s['srm'] < 40:
        return 'Deep Brown'
    elif s['srm'] >= 40:
        return 'Black'
    else:
        return 'Undefined'
    
modifiedjson['srm_category'] = modifiedjson.apply(srmCat, axis=1)

In [6]:
## lk
#function to turn SRM categories into a number
def srmCat2(i):
    if i['srm_category'] == 'Pale Lager, Pilsner, Witbier, Berliner Weisse':
        return 1
#   elif 25 >= i['ibu'] < 40:
    elif i['srm_category'] == 'Straw':
        return 2
    elif i['srm_category'] == 'Pale Gold':
        return 3
    elif i['srm_category'] == 'Deep Gold':
        return 4
    elif i['srm_category'] == 'Pale Amber':
        return 5
    elif i['srm_category'] == 'Medium Amber':
        return 6
    elif i['srm_category'] == 'Amber Brown':
        return 7
    elif i['srm_category'] == 'Deep Amber':
        return 8
    elif i['srm_category'] == 'Brown':
        return 9
    elif i['srm_category'] == 'Ruby Brown':
        return 10
    elif i['srm_category'] == 'Deep Brown':
        return 11
    elif i['srm_category'] == 'Black':
        return 12
    else:
        return 13
    
modifiedjson['srm_category_nbr'] = modifiedjson.apply(srmCat2, axis =1)

In [7]:
#function to scale ibu into 3 categories 
def ibuCat(i):
    if i['ibu'] >= 0 and i['ibu'] < 25:
        return 'IBU Low'
#   elif 25 >= i['ibu'] < 40:
    elif i['ibu'] >= 25 and i['ibu'] < 40:
        return 'IBU Medium'
    elif i['ibu'] >=40:
        return 'IBU High'
    else:
        return 'IBU Undefined'
    
modifiedjson['ibu_category'] = modifiedjson.apply(ibuCat, axis =1)

In [8]:
## lk
#function to turn ibu categories into a number
def ibuCat2(i):
    if i['ibu_category'] == 'IBU Low':
        return 1
#   elif 25 >= i['ibu'] < 40:
    elif i['ibu_category'] == 'IBU Medium':
        return 2
    elif i['ibu_category'] == 'IBU High':
        return 3
    else:
        return 4
    
modifiedjson['ibu_category_nbr'] = modifiedjson.apply(ibuCat2, axis =1)

In [9]:
#function to scale abv into 3 categories 
def abvCat(a):
    if a['abv'] >= 0 and a['abv'] < 9:
        return 'ABV Low'
#   elif 9 >= a['abv'] < 16:
    elif a['abv'] >= 9 and a['abv'] < 16:
        return 'ABV Medium'
    elif a['abv'] >=16:
        return 'ABV High'
    else:
        return 'ABV Undefined'

modifiedjson['abv_category'] = modifiedjson.apply(abvCat, axis =1)   

In [10]:
## lk
#function to turn ibu categories into a number
def abvCat2(a):
    if a['abv_category'] == 'ABV Low':
        return 1
    elif a['abv_category'] == 'ABV Medium':
        return 2
    elif a['abv_category'] == 'ABV High':
        return 3
    else:
        return 4

modifiedjson['abv_category_nbr'] = modifiedjson.apply(abvCat2, axis =1)   

In [11]:
#70-82
def attenCat(a):
    if a['attenuation_level'] >=0.0 and a['attenuation_level'] < 70.0:
        return 'Atten Low'
    elif a['attenuation_level'] >= 70.0 and a['attenuation_level'] < 82.0:
        return 'Atten Medium'
    elif a['attenuation_level'] >= 82.0:
        return 'Atten High'
    else:
        'Undefined'
        
modifiedjson['attenuation_category'] = modifiedjson.apply(attenCat, axis =1)    

In [12]:
## lk
#function to turn attenuation categories into a number
def attenCat2(i):
    if i['attenuation_category'] == 'Atten Low':
        return 1
#   elif 25 >= i['ibu'] < 40:
    elif i['attenuation_category'] == 'Atten Medium':
        return 2
    elif i['attenuation_category'] == 'Atten High':
        return 3
    else:
        return 4
    
modifiedjson['attenuation_category_nbr'] = modifiedjson.apply(attenCat2, axis =1)

In [13]:
beerdata = modifiedjson[['id','name','ibu','ibu_category','ibu_category_nbr','srm','srm_category','srm_category_nbr','abv','abv_category','abv_category_nbr', 'ebc','attenuation_level','attenuation_category','attenuation_category_nbr','description',
                        'image_url','ph', 'tagline','brewers_tips']].set_index('id')
# beerdata.to_csv('beerdata_lk.csv')
beerdata.dtypes

name                         object
ibu                         float64
ibu_category                 object
ibu_category_nbr              int64
srm                         float64
srm_category                 object
srm_category_nbr              int64
abv                         float64
abv_category                 object
abv_category_nbr              int64
ebc                         float64
attenuation_level           float64
attenuation_category         object
attenuation_category_nbr      int64
description                  object
image_url                    object
ph                          float64
tagline                      object
brewers_tips                 object
dtype: object

In [14]:
# combining columns to get a summary outcome column
# data = data2
beerdata['outcome'] = beerdata[['ibu_category','srm_category','abv_category','attenuation_category']].apply(lambda x: ' '.join(x), axis=1)


In [15]:
with open('Resources/mergedjson.json', encoding='utf8') as data_file:
    data = json.load(data_file)

maltdf = json_normalize(data, meta = ['id'], record_path=['ingredients',['malt']],errors='ignore')
# maltdf.drop(columns=['amount'])

maltdf = maltdf.groupby(maltdf['id']).agg(','.join)
maltdf = maltdf.rename(columns={'name': 'malts_name'})
maltdf

hopsdf = json_normalize(data, meta = ['id'], record_path =['ingredients', ['hops']], errors = 'ignore')
# hopsdf.drop(columns=['amount'])
hopsdf = hopsdf.groupby(hopsdf['id']).agg(','.join)
hopsdf = hopsdf.rename(columns={'add':'hops_add', 'attribute':'hops_attribute', 'name': 'hops_name'})

fooddf = json_normalize(data, meta =['id'], record_path=['food_pairing'], errors = 'ignore')
fooddf = fooddf.groupby(fooddf['id']).agg(','.join)
fooddf = fooddf.rename(columns = {0:'food_pairing'})

ingredf = maltdf.merge(hopsdf, how ='outer',left_index=True, right_index=True)
ingredf = ingredf.merge(fooddf, how = 'outer', left_index=True, right_index=True)
finaldata = beerdata.merge(ingredf, how= 'outer', left_index=True, right_index=True)

# finaldata.to_csv('finalbeerdata.csv')


In [16]:
# connection_string = "db/beerdata.sqlite"
# engine = create_engine(f'sqlite:///{connection_string}')
# finaldata.to_sql(name='finaldata', con=engine, if_exists='append', index=True)

In [17]:
# working on decision tree (unit 21, 2-4) - i don't think RandomForest will work
from sklearn import tree
import os


In [18]:
# data2 = data2.drop("ibu_category", axis=1)
data2 = finaldata[['ibu','ibu_category_nbr', 'srm', 'srm_category_nbr', 'abv','abv_category_nbr', 'attenuation_level','attenuation_category_nbr']]
feature_names = data2.columns
data2.dtypes

ibu                         float64
ibu_category_nbr              int64
srm                         float64
srm_category_nbr              int64
abv                         float64
abv_category_nbr              int64
attenuation_level           float64
attenuation_category_nbr      int64
dtype: object

In [19]:
# data2 = data2.drop("ibu_category", axis=1)
# data2 = data2.drop("srm_category", axis=1)
# data2 = data2.drop("abv_category", axis=1)
# data2 = data2.drop("attenuation_level", axis=1)

In [20]:
# data2.to_csv('data2.csv')
data2

,ibu,ibu_category_nbr,srm,srm_category_nbr,abv,abv_category_nbr,attenuation_level,attenuation_category_nbr
id,,,,,,,,
1,60.0,3,10.0,6,4.5,1,75.0,2
2,41.5,3,15.0,8,4.1,1,76.0,2
3,8.0,1,4.0,3,4.2,1,83.0,3
4,55.0,3,15.0,8,6.3,1,80.0,2
5,59.0,3,5.0,3,7.2,1,67.0,1
...,...,...,...,...,...,...,...,...
321,35.0,2,7.0,4,4.7,1,81.0,2
322,65.0,3,9.0,5,6.9,1,87.0,3
323,25.0,2,7.0,4,5.0,1,84.0,3


In [21]:
# data2.mean()

In [22]:
# data2=data2.fillna(data2.mean())
# data2.to_csv('data2.csv')
# data2

#### THIS COMMAND IS TOO LATE AND DOESN'T IMPACT THE CATEGORY NAME OR NBR DOWN HERE.

In [23]:
# data2.to_csv('data2.csv')
# data2

In [24]:
# examples from activity - unit 21/2-04
target = data2["ibu_category_nbr"]
target_names = [1,2,3,4]


In [25]:
test_data = data2.drop("ibu_category_nbr", axis=1)
feature_names = test_data.columns
test_data.head()

,ibu,srm,srm_category_nbr,abv,abv_category_nbr,attenuation_level,attenuation_category_nbr
id,,,,,,,
1,60.0,10.0,6,4.5,1,75.0,2
2,41.5,15.0,8,4.1,1,76.0,2
3,8.0,4.0,3,4.2,1,83.0,3
4,55.0,15.0,8,6.3,1,80.0,2
5,59.0,5.0,3,7.2,1,67.0,1


In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(test_data, target, random_state=42)

In [27]:
# accuracy of predicting IBU Category
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

In [28]:
# examples from activity - unit 21/2-04
target2 = data2["abv_category_nbr"]
target_names2 = [1,2,3,4]

In [29]:
test_data2 = data2.drop("abv_category_nbr", axis=1)
feature_names2 = test_data2.columns
test_data2.head()

,ibu,ibu_category_nbr,srm,srm_category_nbr,abv,attenuation_level,attenuation_category_nbr
id,,,,,,,
1,60.0,3,10.0,6,4.5,75.0,2
2,41.5,3,15.0,8,4.1,76.0,2
3,8.0,1,4.0,3,4.2,83.0,3
4,55.0,3,15.0,8,6.3,80.0,2
5,59.0,3,5.0,3,7.2,67.0,1


In [30]:
# from sklearn.model_selection import train_test_split (already brought in)
X_train, X_test, y_train, y_test = train_test_split(test_data2, target2, random_state=42)

In [31]:
# accuracy of predicting ABV Category
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

In [32]:
# examples from activity - unit 21/2-04
target3 = data2["attenuation_category_nbr"]
target_names3 = [1,2,3,4]

In [33]:
test_data3 = data2.drop("attenuation_category_nbr", axis=1)
feature_names3 = test_data3.columns
test_data3.head()

,ibu,ibu_category_nbr,srm,srm_category_nbr,abv,abv_category_nbr,attenuation_level
id,,,,,,,
1,60.0,3,10.0,6,4.5,1,75.0
2,41.5,3,15.0,8,4.1,1,76.0
3,8.0,1,4.0,3,4.2,1,83.0
4,55.0,3,15.0,8,6.3,1,80.0
5,59.0,3,5.0,3,7.2,1,67.0


In [34]:
# from sklearn.model_selection import train_test_split (already brought in)
X_train, X_test, y_train, y_test = train_test_split(test_data3, target3, random_state=42)

In [35]:
# accuracy of predicting Attenuation Category
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

In [36]:
# examples from activity - unit 21/2-04
target4 = data2["srm_category_nbr"]
target_names4 = [1,2,3,4]

In [37]:
test_data4 = data2.drop("srm_category_nbr", axis=1)
feature_names4 = test_data4.columns
test_data4.head()

,ibu,ibu_category_nbr,srm,abv,abv_category_nbr,attenuation_level,attenuation_category_nbr
id,,,,,,,
1,60.0,3,10.0,4.5,1,75.0,2
2,41.5,3,15.0,4.1,1,76.0,2
3,8.0,1,4.0,4.2,1,83.0,3
4,55.0,3,15.0,6.3,1,80.0,2
5,59.0,3,5.0,7.2,1,67.0,1


In [38]:
# from sklearn.model_selection import train_test_split (already brought in)
X_train, X_test, y_train, y_test = train_test_split(test_data3, target3, random_state=42)

In [39]:
# accuracy of predicting Attenuation Category
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train, y_train)
clf.score(X_test, y_test)

1.0

In [40]:
## **** Applying One-Hot Encoding ***** ##

In [41]:
### trying a different approach - using activity 21-3-1
# learning2 = finaldata[['ibu','srm', 'abv','attenuation_level','outcome']]
learning2 = finaldata[['ibu_category_nbr','srm_category_nbr', 'abv_category_nbr','attenuation_category_nbr','outcome']]

feature_names = learning2.columns
# learning2.to_csv('learning2.csv')
learning2.dtypes



ibu_category_nbr             int64
srm_category_nbr             int64
abv_category_nbr             int64
attenuation_category_nbr     int64
outcome                     object
dtype: object

In [42]:
len(learning2['outcome'].unique())

81

In [43]:
# learning2['predicted_class'] = learning2.groupby(learning2.columns.tolist(), sort=False).ngroup() + 1
# learning2



# df['new_id'] = df.groupby(df.columns.tolist(), sort=False).ngroup() + 1
# df = df.assign(id=(df['LastName'] + '_' + df['FirstName']).astype('category').cat.codes)

In [44]:
feature_names = learning2.columns
# learning2.to_csv('learning2.csv')
learning2.dtypes

ibu_category_nbr             int64
srm_category_nbr             int64
abv_category_nbr             int64
attenuation_category_nbr     int64
outcome                     object
dtype: object

In [45]:
learning2.head()

,ibu_category_nbr,srm_category_nbr,abv_category_nbr,attenuation_category_nbr,outcome
id,,,,,
1,3,6,1,2,IBU High Medium Amber ABV Low Atten Medium
2,3,8,1,2,IBU High Deep Amber ABV Low Atten Medium
3,1,3,1,3,IBU Low Pale Gold ABV Low Atten High
4,3,8,1,2,IBU High Deep Amber ABV Low Atten Medium
5,3,3,1,1,IBU High Pale Gold ABV Low Atten Low


In [46]:
# applying One-Hot Enconding
# Step 0: Reformat data
data = learning2.values
X = data[:, 0:4]
# y = data[:, 4]
y = data[:, 4].reshape(-1,1)
print(X.shape,y.shape)

(325, 4) (325, 1)


In [47]:
from sklearn.preprocessing import LabelEncoder

# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y)
encoded_y = label_encoder.transform(y)

list(label_encoder.classes_)
label_encoder.transform(list(label_encoder.classes_))

C:\Users\lonny\anacond\Anaconda3\envs\project3\lib\site-packages\sklearn\preprocessing\label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\lonny\anacond\Anaconda3\envs\project3\lib\site-packages\sklearn\preprocessing\label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
       68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80], dtype=int64)

In [48]:
for label, original_class in zip(encoded_y, y):
    print('Original Class: ' + str(original_class))
    print('Encoded Label: ' + str(label))
    print('-' * 12)
#     learning2.append("predicted_class",label)

Original Class: ['IBU High Medium Amber ABV Low Atten Medium']
Encoded Label: 28
------------
Original Class: ['IBU High Deep Amber ABV Low Atten Medium']
Encoded Label: 18
------------
Original Class: ['IBU Low Pale Gold ABV Low Atten High']
Encoded Label: 50
------------
Original Class: ['IBU High Deep Amber ABV Low Atten Medium']
Encoded Label: 18
------------
Original Class: ['IBU High Pale Gold ABV Low Atten Low']
Encoded Label: 35
------------
Original Class: ['IBU Medium Deep Gold ABV Low Atten High']
Encoded Label: 67
------------
Original Class: ['IBU Medium Black ABV Medium Atten High']
Encoded Label: 61
------------
Original Class: ['IBU High Deep Gold ABV Low Atten Medium']
Encoded Label: 25
------------
Original Class: ['IBU Medium Black ABV Medium Atten High']
Encoded Label: 61
------------
Original Class: ['IBU High Medium Amber ABV Low Atten Medium']
Encoded Label: 28
------------
Original Class: ['IBU Medium Black ABV Low Atten Medium']
Encoded Label: 60
------------
O

In [49]:
# Note that each of the original labels has been replaced with an integer.
from keras.utils import to_categorical

# Step 2: One-hot encoding
one_hot_y = to_categorical(encoded_y)
np.savetxt("one_hot_y.csv",one_hot_y, delimiter=",")
one_hot_y

Using TensorFlow backend.
C:\Users\lonny\anacond\Anaconda3\envs\project3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\lonny\anacond\Anaconda3\envs\project3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\lonny\anacond\Anaconda3\envs\project3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\lonny

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.]], dtype=float32)

In [50]:
encoded_y

array([28, 18, 50, 18, 35, 67, 61, 25, 61, 28, 60,  9,  9, 39, 14,  9, 23,
       28, 38, 72, 66,  3, 24, 37, 36, 78, 10, 32,  9, 18, 17, 11, 16, 63,
       54, 40, 15, 18, 16, 31, 11, 15, 52, 24, 39, 46, 11, 18, 72, 61, 44,
       10, 65, 38, 16, 11, 33, 11, 24, 65, 68, 29, 24, 68,  9, 10, 75, 32,
       19, 11, 65, 45, 11, 32, 71, 13, 18, 11, 39, 10, 68, 27, 16, 53, 65,
       16, 71, 18, 16, 11, 71, 11, 34, 26, 32, 42, 30, 43, 16, 27,  9, 69,
       29,  0,  7, 25, 16, 18, 20, 79, 28, 52, 21,  9, 34, 24, 28, 60, 60,
       11, 31, 16, 13, 25, 24,  9, 32, 18, 11, 54, 32, 79,  7,  9, 74, 29,
       12, 10, 24, 24, 10, 57, 71,  5, 24, 16, 66, 37, 56, 11, 75, 32, 34,
       76, 11, 16, 79, 28,  6, 24, 10, 68, 67, 24, 41, 28,  9, 24, 41, 19,
       32, 36, 24, 16, 11, 38,  5, 18,  1, 76, 16, 64, 74, 16,  5,  1, 32,
       10, 11, 16, 39, 31, 50, 13, 71,  9,  1, 49,  9,  0, 18, 10,  2, 24,
       22, 26, 55, 24,  9, 60,  8, 25, 18, 31,  9, 11, 40, 24, 19, 32, 58,
        5, 76, 56, 10, 24

In [51]:
# from activity 21-3-2

In [52]:
# from sklearn.model_selection import train_test_split (already brought in)
X_train, X_test, y_train, y_test = train_test_split(X, one_hot_y, random_state=42)

In [53]:
# Data Preprocessing
# It is really important to scale our data before using multilayer perceptron models.
# Without scaling, it is often difficult for the training cycle to converge
from sklearn.preprocessing import StandardScaler

X_scaler = StandardScaler().fit(X_train)


In [54]:
# Remember to scale both the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [55]:
# One-hot encoding - already done a few cells above 
# y_train_categorical = to_categorical(y_train)
# y_test_categorical = to_categorical(y_test)
# y_train_categorical
y_train_categorical=y_train

In [56]:
y_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [57]:
# Defining our Model Architecture (the layers)
# We first need to create a sequential model

from tensorflow.keras.models import Sequential

model = Sequential()

In [58]:
# Next, we add our first layer. This layer requires you to specify both the number of inputs and 
# the number of nodes that you want in the hidden layer.
from tensorflow.keras.layers import Dense
number_inputs = 4    #this is features
number_hidden_nodes = 4
model.add(Dense(units=number_hidden_nodes,
                activation='relu', input_dim=number_inputs))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [59]:
# Our final layer is the output layer. Here, we need to specify the activation function 
# (typically softmax for classification) and the number of classes (labels) that we are trying to 
# predict (81 in this learning2 - Viewer).

number_classes = len(learning2['outcome'].unique())
model.add(Dense(units=number_classes, activation='softmax'))

In [60]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 20        
_________________________________________________________________
dense_1 (Dense)              (None, 81)                405       
Total params: 425
Trainable params: 425
Non-trainable params: 0
_________________________________________________________________


In [61]:
# Compile the Model
# Now that we have our model architecture defined, we must compile the model using a loss 
# function and optimizer. We can also specify additional training metrics such as accuracy.

In [62]:
# Use categorical crossentropy for categorical data and mean squared error for regression
# Hint: your output layer in this example is using software for logistic regression (categorical)
# If your output layer activation was `linear` then you may want to use `mse` for loss
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [63]:
# Training the Model
# Finally, we train our model using our training data

# Training consists of updating our weights using our optimizer and loss function. In this 
# example, we choose 1000 iterations (loops) of training that are called epochs.

# We also choose to shuffle our training data and increase the detail printed out during each 
# training cycle.

# Fit (train) the model
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=1000,
    batch_size=25,
    shuffle=True,
    verbose=2
)

Epoch 1/1000
243/243 - 0s - loss: 4.4034 - acc: 0.0082
Epoch 2/1000
243/243 - 0s - loss: 4.3805 - acc: 0.0082
Epoch 3/1000
243/243 - 0s - loss: 4.3599 - acc: 0.0576
Epoch 4/1000
243/243 - 0s - loss: 4.3389 - acc: 0.1193
Epoch 5/1000
243/243 - 0s - loss: 4.3184 - acc: 0.1193
Epoch 6/1000
243/243 - 0s - loss: 4.2979 - acc: 0.1193
Epoch 7/1000
243/243 - 0s - loss: 4.2770 - acc: 0.1193
Epoch 8/1000
243/243 - 0s - loss: 4.2560 - acc: 0.1193
Epoch 9/1000
243/243 - 0s - loss: 4.2342 - acc: 0.1193
Epoch 10/1000
243/243 - 0s - loss: 4.2125 - acc: 0.1276
Epoch 11/1000
243/243 - 0s - loss: 4.1904 - acc: 0.1317
Epoch 12/1000
243/243 - 0s - loss: 4.1676 - acc: 0.1317
Epoch 13/1000
243/243 - 0s - loss: 4.1444 - acc: 0.1276
Epoch 14/1000
243/243 - 0s - loss: 4.1202 - acc: 0.1193
Epoch 15/1000
243/243 - 0s - loss: 4.0969 - acc: 0.1193
Epoch 16/1000
243/243 - 0s - loss: 4.0715 - acc: 0.1193
Epoch 17/1000
243/243 - 0s - loss: 4.0467 - acc: 0.1193
Epoch 18/1000
243/243 - 0s - loss: 4.0202 - acc: 0.1193
E

Epoch 147/1000
243/243 - 0s - loss: 2.0812 - acc: 0.4115
Epoch 148/1000
243/243 - 0s - loss: 2.0733 - acc: 0.4033
Epoch 149/1000
243/243 - 0s - loss: 2.0652 - acc: 0.3992
Epoch 150/1000
243/243 - 0s - loss: 2.0575 - acc: 0.4033
Epoch 151/1000
243/243 - 0s - loss: 2.0499 - acc: 0.4115
Epoch 152/1000
243/243 - 0s - loss: 2.0425 - acc: 0.4115
Epoch 153/1000
243/243 - 0s - loss: 2.0348 - acc: 0.4115
Epoch 154/1000
243/243 - 0s - loss: 2.0272 - acc: 0.4115
Epoch 155/1000
243/243 - 0s - loss: 2.0199 - acc: 0.4115
Epoch 156/1000
243/243 - 0s - loss: 2.0124 - acc: 0.4280
Epoch 157/1000
243/243 - 0s - loss: 2.0051 - acc: 0.4280
Epoch 158/1000
243/243 - 0s - loss: 1.9979 - acc: 0.4280
Epoch 159/1000
243/243 - 0s - loss: 1.9909 - acc: 0.4280
Epoch 160/1000
243/243 - 0s - loss: 1.9835 - acc: 0.4280
Epoch 161/1000
243/243 - 0s - loss: 1.9763 - acc: 0.4280
Epoch 162/1000
243/243 - 0s - loss: 1.9696 - acc: 0.4280
Epoch 163/1000
243/243 - 0s - loss: 1.9626 - acc: 0.4280
Epoch 164/1000
243/243 - 0s - l

Epoch 291/1000
243/243 - 0s - loss: 1.3269 - acc: 0.6667
Epoch 292/1000
243/243 - 0s - loss: 1.3236 - acc: 0.6667
Epoch 293/1000
243/243 - 0s - loss: 1.3196 - acc: 0.6667
Epoch 294/1000
243/243 - 0s - loss: 1.3162 - acc: 0.6667
Epoch 295/1000
243/243 - 0s - loss: 1.3126 - acc: 0.6667
Epoch 296/1000
243/243 - 0s - loss: 1.3094 - acc: 0.6667
Epoch 297/1000
243/243 - 0s - loss: 1.3059 - acc: 0.6667
Epoch 298/1000
243/243 - 0s - loss: 1.3026 - acc: 0.6667
Epoch 299/1000
243/243 - 0s - loss: 1.2984 - acc: 0.6667
Epoch 300/1000
243/243 - 0s - loss: 1.2958 - acc: 0.6667
Epoch 301/1000
243/243 - 0s - loss: 1.2919 - acc: 0.6667
Epoch 302/1000
243/243 - 0s - loss: 1.2881 - acc: 0.6667
Epoch 303/1000
243/243 - 0s - loss: 1.2849 - acc: 0.6667
Epoch 304/1000
243/243 - 0s - loss: 1.2816 - acc: 0.6708
Epoch 305/1000
243/243 - 0s - loss: 1.2781 - acc: 0.6831
Epoch 306/1000
243/243 - 0s - loss: 1.2748 - acc: 0.6831
Epoch 307/1000
243/243 - 0s - loss: 1.2717 - acc: 0.6831
Epoch 308/1000
243/243 - 0s - l

Epoch 435/1000
243/243 - 0s - loss: 0.9171 - acc: 0.8395
Epoch 436/1000
243/243 - 0s - loss: 0.9155 - acc: 0.8395
Epoch 437/1000
243/243 - 0s - loss: 0.9131 - acc: 0.8395
Epoch 438/1000
243/243 - 0s - loss: 0.9108 - acc: 0.8354
Epoch 439/1000
243/243 - 0s - loss: 0.9089 - acc: 0.8313
Epoch 440/1000
243/243 - 0s - loss: 0.9067 - acc: 0.8313
Epoch 441/1000
243/243 - 0s - loss: 0.9042 - acc: 0.8354
Epoch 442/1000
243/243 - 0s - loss: 0.9023 - acc: 0.8313
Epoch 443/1000
243/243 - 0s - loss: 0.9000 - acc: 0.8354
Epoch 444/1000
243/243 - 0s - loss: 0.8982 - acc: 0.8313
Epoch 445/1000
243/243 - 0s - loss: 0.8958 - acc: 0.8313
Epoch 446/1000
243/243 - 0s - loss: 0.8941 - acc: 0.8354
Epoch 447/1000
243/243 - 0s - loss: 0.8924 - acc: 0.8395
Epoch 448/1000
243/243 - 0s - loss: 0.8901 - acc: 0.8395
Epoch 449/1000
243/243 - 0s - loss: 0.8882 - acc: 0.8313
Epoch 450/1000
243/243 - 0s - loss: 0.8864 - acc: 0.8395
Epoch 451/1000
243/243 - 0s - loss: 0.8842 - acc: 0.8313
Epoch 452/1000
243/243 - 0s - l

Epoch 579/1000
243/243 - 0s - loss: 0.6949 - acc: 0.8395
Epoch 580/1000
243/243 - 0s - loss: 0.6936 - acc: 0.8395
Epoch 581/1000
243/243 - 0s - loss: 0.6924 - acc: 0.8477
Epoch 582/1000
243/243 - 0s - loss: 0.6915 - acc: 0.8477
Epoch 583/1000
243/243 - 0s - loss: 0.6906 - acc: 0.8477
Epoch 584/1000
243/243 - 0s - loss: 0.6892 - acc: 0.8477
Epoch 585/1000
243/243 - 0s - loss: 0.6880 - acc: 0.8477
Epoch 586/1000
243/243 - 0s - loss: 0.6876 - acc: 0.8477
Epoch 587/1000
243/243 - 0s - loss: 0.6859 - acc: 0.8477
Epoch 588/1000
243/243 - 0s - loss: 0.6845 - acc: 0.8477
Epoch 589/1000
243/243 - 0s - loss: 0.6835 - acc: 0.8477
Epoch 590/1000
243/243 - 0s - loss: 0.6827 - acc: 0.8477
Epoch 591/1000
243/243 - 0s - loss: 0.6817 - acc: 0.8477
Epoch 592/1000
243/243 - 0s - loss: 0.6809 - acc: 0.8477
Epoch 593/1000
243/243 - 0s - loss: 0.6795 - acc: 0.8477
Epoch 594/1000
243/243 - 0s - loss: 0.6783 - acc: 0.8477
Epoch 595/1000
243/243 - 0s - loss: 0.6777 - acc: 0.8395
Epoch 596/1000
243/243 - 0s - l

Epoch 723/1000
243/243 - 0s - loss: 0.5632 - acc: 0.8683
Epoch 724/1000
243/243 - 0s - loss: 0.5620 - acc: 0.8642
Epoch 725/1000
243/243 - 0s - loss: 0.5616 - acc: 0.8724
Epoch 726/1000
243/243 - 0s - loss: 0.5605 - acc: 0.8683
Epoch 727/1000
243/243 - 0s - loss: 0.5595 - acc: 0.8642
Epoch 728/1000
243/243 - 0s - loss: 0.5590 - acc: 0.8642
Epoch 729/1000
243/243 - 0s - loss: 0.5583 - acc: 0.8683
Epoch 730/1000
243/243 - 0s - loss: 0.5576 - acc: 0.8683
Epoch 731/1000
243/243 - 0s - loss: 0.5566 - acc: 0.8683
Epoch 732/1000
243/243 - 0s - loss: 0.5559 - acc: 0.8683
Epoch 733/1000
243/243 - 0s - loss: 0.5551 - acc: 0.8683
Epoch 734/1000
243/243 - 0s - loss: 0.5542 - acc: 0.8683
Epoch 735/1000
243/243 - 0s - loss: 0.5535 - acc: 0.8683
Epoch 736/1000
243/243 - 0s - loss: 0.5530 - acc: 0.8683
Epoch 737/1000
243/243 - 0s - loss: 0.5523 - acc: 0.8683
Epoch 738/1000
243/243 - 0s - loss: 0.5513 - acc: 0.8683
Epoch 739/1000
243/243 - 0s - loss: 0.5508 - acc: 0.8683
Epoch 740/1000
243/243 - 0s - l

Epoch 867/1000
243/243 - 0s - loss: 0.4677 - acc: 0.8807
Epoch 868/1000
243/243 - 0s - loss: 0.4665 - acc: 0.8765
Epoch 869/1000
243/243 - 0s - loss: 0.4663 - acc: 0.8807
Epoch 870/1000
243/243 - 0s - loss: 0.4649 - acc: 0.8807
Epoch 871/1000
243/243 - 0s - loss: 0.4649 - acc: 0.8807
Epoch 872/1000
243/243 - 0s - loss: 0.4638 - acc: 0.8807
Epoch 873/1000
243/243 - 0s - loss: 0.4636 - acc: 0.8807
Epoch 874/1000
243/243 - 0s - loss: 0.4630 - acc: 0.8807
Epoch 875/1000
243/243 - 0s - loss: 0.4625 - acc: 0.8807
Epoch 876/1000
243/243 - 0s - loss: 0.4615 - acc: 0.8807
Epoch 877/1000
243/243 - 0s - loss: 0.4609 - acc: 0.8807
Epoch 878/1000
243/243 - 0s - loss: 0.4609 - acc: 0.8807
Epoch 879/1000
243/243 - 0s - loss: 0.4600 - acc: 0.8807
Epoch 880/1000
243/243 - 0s - loss: 0.4593 - acc: 0.8807
Epoch 881/1000
243/243 - 0s - loss: 0.4584 - acc: 0.8807
Epoch 882/1000
243/243 - 0s - loss: 0.4582 - acc: 0.8807
Epoch 883/1000
243/243 - 0s - loss: 0.4573 - acc: 0.8807
Epoch 884/1000
243/243 - 0s - l

In [64]:
# Quantifying the Model
# We use our testing data to validate our model. This is how we determine the validity of our 
# model (i.e. the ability to predict new and previously unseen data points)

In [65]:
# Evaluate the model using the testing data
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

82/82 - 0s - loss: 2.9033 - acc: 0.8293
Loss: 2.9032977993895366, Accuracy: 0.8292682766914368


In [66]:
model.save("decision_tree_model.h5")

In [88]:
!ls -l


total 4576
-rw-r--r-- 1 lonny 197609  194915 Jan  4 21:10 abv_ibu_test.ipynb
-rw-r--r-- 1 lonny 197609    2390 Jan  7 19:48 app.py
-rw-r--r-- 1 lonny 197609  677652 Dec 31 10:27 beerdata.csv
-rw-r--r-- 1 lonny 197609       0 Jan  7 19:29 beerdata_table.html
-rw-r--r-- 1 lonny 197609 1288070 Jan  7 19:31 cleandata.ipynb
-rw-r--r-- 1 lonny 197609  139843 Jan  4 21:10 datacleantodb.ipynb
-rw-r--r-- 1 lonny 197609  247674 Jan  7 20:10 datacleantodb-Copy1.ipynb
drwxr-xr-x 1 lonny 197609       0 Jan  6 15:10 db
-rw-r--r-- 1 lonny 197609   29640 Jan  7 20:11 decision_tree_model.h5
-rw-r--r-- 1 lonny 197609   42598 Dec 26 12:39 IBU by Beer.png
-rw-r--r-- 1 lonny 197609  682110 Dec 21 12:49 merged_file.json
-rw-r--r-- 1 lonny 197609  658450 Jan  7 19:41 one_hot_y.csv
-rw-r--r-- 1 lonny 197609  302156 Jan  6 14:49 output_df - Viewer.csv
-rw-r--r-- 1 lonny 197609  304480 Jan  7 16:55 output_df.csv
drwxr-xr-x 1 lonny 197609       0 Jan  4 21:10 Resources
-rw-r--r-- 1 lonny 197609       0 Dec 23 14

In [89]:
# Making Predictions with new data
# We can use our trained model to make predictions using model.predict
import tensorflow as tf
saved_model = tf.keras.models.load_model('decision_tree_model.h5')
new_data = np.array([[1, 1, 1, 1]])
str(saved_model.predict_classes(new_data))

'[5]'

In [91]:
# import numpy as np (already brought in)
# [ibu, srm, abv, attenuation]
new_data = np.array([[3, 11, 3, 3]])
print(f"Predicted class: {model.predict_classes(new_data)+1}")

Predicted class: [11]


In [92]:
label_encoder.inverse_transform([11])

array(['IBU High Black ABV Medium Atten Medium'], dtype=object)

In [69]:
# [ibu, srm, abv, attenuation]
new_data = np.array([[0, 0, 0, 0]])
print(f"Predicted class: {model.predict_classes(new_data)}")

Predicted class: [29]


In [70]:
# [ibu, srm, abv, attenuation]
new_data = np.array([[5, 5, 2, 10]])
print(f"Predicted class: {model.predict_classes(new_data)}")

Predicted class: [7]


In [71]:
# [ibu, srm, abv, attenuation]
new_data = np.array([[2, 10, 3, 1]])
print(f"Predicted class: {model.predict_classes(new_data)}")

Predicted class: [10]


In [85]:
# [ibu, srm, abv, attenuation]
new_data = np.array([[1, 1, 1, 1]])
print(f"Predicted class: {model.predict_classes(new_data)}")

Predicted class: [40]


In [75]:
# [ibu, srm, abv, attenuation]
new_data = np.array([[2, 2, 2, 2]])
print(new_data.shape)
print(f"Predicted class: {model.predict_classes(new_data)}")

(1, 4)
Predicted class: [10]


In [77]:
# [ibu, srm, abv, attenuation]
new_data = np.array([[10,30,10,10]])
x=model.predict_classes(new_data)
print(x.shape)
print(f"Predicted class: {model.predict_classes(new_data)}")


(1,)
Predicted class: [10]


In [143]:
## Grab just one data point to test 
test=np.expand_dims(X_test_scaled[1], axis=0)
test.shape

(1, 4)

In [150]:
X_test[0]

array([3, 12, 2, 3], dtype=object)

In [151]:
X_test_scaled[0]

array([0.61523519, 1.30610515, 1.28570191, 0.97974488])

In [141]:
## make a prediction
print(f"Predicted Class: {model.predict_classes(test)}")

Predicted Class: [16]


In [132]:
# merge the original df and the learning2 df to allow for grabbing full info
output_df = finaldata.merge(learning2, on='id', how='left')
output_df.head()

,name,ibu,ibu_category,ibu_category_nbr_x,srm,srm_category,srm_category_nbr_x,abv,abv_category,abv_category_nbr_x,...,hops_name,hops_add,hops_attribute,amount.unit_y,food_pairing,ibu_category_nbr_y,srm_category_nbr_y,abv_category_nbr_y,attenuation_category_nbr_y,outcome_y
id,,,,,,,,,,,,,,,,,,,,,
1,Buzz,60.0,IBU High,3,10.0,Medium Amber,6,4.5,ABV Low,1,...,"Fuggles,First Gold,Fuggles,First Gold,Cascade","start,start,middle,middle,end","bitter,bitter,flavour,flavour,flavour","grams,grams,grams,grams,grams","Spicy chicken tikka masala,Grilled chicken que...",3,6,1,2,IBU High Medium Amber ABV Low Atten Medium
2,Trashy Blonde,41.5,IBU High,3,15.0,Deep Amber,8,4.1,ABV Low,1,...,"Amarillo,Simcoe,Amarillo,Motueka","start,start,end,end","bitter,bitter,flavour,flavour","grams,grams,grams,grams","Fresh crab with lemon,Garlic butter dipping sa...",3,8,1,2,IBU High Deep Amber ABV Low Atten Medium
3,Berliner Weisse With Yuzu - B-Sides,8.0,IBU Low,1,4.0,Pale Gold,3,4.2,ABV Low,1,...,Bramling Cross,middle,bitter,grams,"Smoked chicken wings,Miso ramen,Yuzu cheesecake",1,3,1,3,IBU Low Pale Gold ABV Low Atten High
4,Pilsen Lager,55.0,IBU High,3,15.0,Deep Amber,8,6.3,ABV Low,1,...,"Centennial,Amarillo,Centennial,Amarillo,Centen...","start,start,middle,middle,end,end","bitter,bitter,flavour,flavour,flavour,flavour","grams,grams,grams,grams,grams,grams","Spicy crab cakes,Spicy cucumber and carrot Tha...",3,8,1,2,IBU High Deep Amber ABV Low Atten Medium
5,Avery Brown Dredge,59.0,IBU High,3,5.0,Pale Gold,3,7.2,ABV Low,1,...,"Saaz,Saaz,Saaz","start,middle,end","bitter,flavour,flavour","grams,grams,grams","Vietnamese squid salad,Chargrilled corn on the...",3,3,1,1,IBU High Pale Gold ABV Low Atten Low


In [100]:
output_df.drop(['ibu_category_nbr_y', 'srm_category_nbr_y', 
                'abv_category_nbr_y', 'attenuation_category_nbr_y','outcome_y'], axis=1, inplace=True)

In [101]:
output_df.head()

,name,ibu,ibu_category,ibu_category_nbr_x,srm,srm_category,srm_category_nbr_x,abv,abv_category,abv_category_nbr_x,...,tagline,brewers_tips,outcome_x,malts_name,amount.unit_x,hops_name,hops_add,hops_attribute,amount.unit_y,food_pairing
id,,,,,,,,,,,,,,,,,,,,,
1,Buzz,60.0,IBU High,3,10.0,Medium Amber,6,4.5,ABV Low,1,...,A Real Bitter Experience.,The earthy and floral aromas from the hops can...,IBU High Medium Amber ABV Low Atten Medium,"Maris Otter Extra Pale,Caramalt,Munich","kilograms,kilograms,kilograms","Fuggles,First Gold,Fuggles,First Gold,Cascade","start,start,middle,middle,end","bitter,bitter,flavour,flavour,flavour","grams,grams,grams,grams,grams","Spicy chicken tikka masala,Grilled chicken que..."
2,Trashy Blonde,41.5,IBU High,3,15.0,Deep Amber,8,4.1,ABV Low,1,...,You Know You Shouldn't,Be careful not to collect too much wort from t...,IBU High Deep Amber ABV Low Atten Medium,"Maris Otter Extra Pale,Caramalt,Munich","kilograms,kilograms,kilograms","Amarillo,Simcoe,Amarillo,Motueka","start,start,end,end","bitter,bitter,flavour,flavour","grams,grams,grams,grams","Fresh crab with lemon,Garlic butter dipping sa..."
3,Berliner Weisse With Yuzu - B-Sides,8.0,IBU Low,1,4.0,Pale Gold,3,4.2,ABV Low,1,...,Japanese Citrus Berliner Weisse.,Clean everything twice. All you want is the cl...,IBU Low Pale Gold ABV Low Atten High,"Propino Pale Malt,Wheat Malt,Propino Pale Malt...","kilograms,kilograms,kilograms,kilograms",Bramling Cross,middle,bitter,grams,"Smoked chicken wings,Miso ramen,Yuzu cheesecake"
4,Pilsen Lager,55.0,IBU High,3,15.0,Deep Amber,8,6.3,ABV Low,1,...,Unleash the Yeast Series.,Play around with the fermentation temperature ...,IBU High Deep Amber ABV Low Atten Medium,"Extra Pale,Caramalt,Dark Crystal,Munich","kilograms,kilograms,kilograms,kilograms","Centennial,Amarillo,Centennial,Amarillo,Centen...","start,start,middle,middle,end,end","bitter,bitter,flavour,flavour,flavour,flavour","grams,grams,grams,grams,grams,grams","Spicy crab cakes,Spicy cucumber and carrot Tha..."
5,Avery Brown Dredge,59.0,IBU High,3,5.0,Pale Gold,3,7.2,ABV Low,1,...,Bloggers' Imperial Pilsner.,Make sure you have a big enough yeast starter ...,IBU High Pale Gold ABV Low Atten Low,"Lager Malt,Wheat","kilograms,kilograms","Saaz,Saaz,Saaz","start,middle,end","bitter,flavour,flavour","grams,grams,grams","Vietnamese squid salad,Chargrilled corn on the..."


In [102]:
output_df=output_df.rename(columns={
    'ibu_category_nbr_x':'ibu_category_nbr',
    'srm_category_nbr_x':'srm_category_nbr',
    'abv_category_nbr_x':'abv_category_nbr',
    'attenuation_category_nbr_x':'attenuation_category_nbr',
    'outcome_x':'outcome',
    'amount.unit_x':'amount_unit_x',
    'amount.unit_y':'amount_unit_y'})


In [103]:
output_df.head()

,name,ibu,ibu_category,ibu_category_nbr,srm,srm_category,srm_category_nbr,abv,abv_category,abv_category_nbr,...,tagline,brewers_tips,outcome,malts_name,amount_unit_x,hops_name,hops_add,hops_attribute,amount_unit_y,food_pairing
id,,,,,,,,,,,,,,,,,,,,,
1,Buzz,60.0,IBU High,3,10.0,Medium Amber,6,4.5,ABV Low,1,...,A Real Bitter Experience.,The earthy and floral aromas from the hops can...,IBU High Medium Amber ABV Low Atten Medium,"Maris Otter Extra Pale,Caramalt,Munich","kilograms,kilograms,kilograms","Fuggles,First Gold,Fuggles,First Gold,Cascade","start,start,middle,middle,end","bitter,bitter,flavour,flavour,flavour","grams,grams,grams,grams,grams","Spicy chicken tikka masala,Grilled chicken que..."
2,Trashy Blonde,41.5,IBU High,3,15.0,Deep Amber,8,4.1,ABV Low,1,...,You Know You Shouldn't,Be careful not to collect too much wort from t...,IBU High Deep Amber ABV Low Atten Medium,"Maris Otter Extra Pale,Caramalt,Munich","kilograms,kilograms,kilograms","Amarillo,Simcoe,Amarillo,Motueka","start,start,end,end","bitter,bitter,flavour,flavour","grams,grams,grams,grams","Fresh crab with lemon,Garlic butter dipping sa..."
3,Berliner Weisse With Yuzu - B-Sides,8.0,IBU Low,1,4.0,Pale Gold,3,4.2,ABV Low,1,...,Japanese Citrus Berliner Weisse.,Clean everything twice. All you want is the cl...,IBU Low Pale Gold ABV Low Atten High,"Propino Pale Malt,Wheat Malt,Propino Pale Malt...","kilograms,kilograms,kilograms,kilograms",Bramling Cross,middle,bitter,grams,"Smoked chicken wings,Miso ramen,Yuzu cheesecake"
4,Pilsen Lager,55.0,IBU High,3,15.0,Deep Amber,8,6.3,ABV Low,1,...,Unleash the Yeast Series.,Play around with the fermentation temperature ...,IBU High Deep Amber ABV Low Atten Medium,"Extra Pale,Caramalt,Dark Crystal,Munich","kilograms,kilograms,kilograms,kilograms","Centennial,Amarillo,Centennial,Amarillo,Centen...","start,start,middle,middle,end,end","bitter,bitter,flavour,flavour,flavour,flavour","grams,grams,grams,grams,grams,grams","Spicy crab cakes,Spicy cucumber and carrot Tha..."
5,Avery Brown Dredge,59.0,IBU High,3,5.0,Pale Gold,3,7.2,ABV Low,1,...,Bloggers' Imperial Pilsner.,Make sure you have a big enough yeast starter ...,IBU High Pale Gold ABV Low Atten Low,"Lager Malt,Wheat","kilograms,kilograms","Saaz,Saaz,Saaz","start,middle,end","bitter,flavour,flavour","grams,grams,grams","Vietnamese squid salad,Chargrilled corn on the..."


In [90]:
label_encoder.inverse_transform([5])

array(['IBU High Black ABV High Atten High'], dtype=object)

In [105]:
output_df['ai_prediction'] = encoded_y

In [106]:
output_df.head()

,name,ibu,ibu_category,ibu_category_nbr,srm,srm_category,srm_category_nbr,abv,abv_category,abv_category_nbr,...,brewers_tips,outcome,malts_name,amount_unit_x,hops_name,hops_add,hops_attribute,amount_unit_y,food_pairing,ai_prediction
id,,,,,,,,,,,,,,,,,,,,,
1,Buzz,60.0,IBU High,3,10.0,Medium Amber,6,4.5,ABV Low,1,...,The earthy and floral aromas from the hops can...,IBU High Medium Amber ABV Low Atten Medium,"Maris Otter Extra Pale,Caramalt,Munich","kilograms,kilograms,kilograms","Fuggles,First Gold,Fuggles,First Gold,Cascade","start,start,middle,middle,end","bitter,bitter,flavour,flavour,flavour","grams,grams,grams,grams,grams","Spicy chicken tikka masala,Grilled chicken que...",28
2,Trashy Blonde,41.5,IBU High,3,15.0,Deep Amber,8,4.1,ABV Low,1,...,Be careful not to collect too much wort from t...,IBU High Deep Amber ABV Low Atten Medium,"Maris Otter Extra Pale,Caramalt,Munich","kilograms,kilograms,kilograms","Amarillo,Simcoe,Amarillo,Motueka","start,start,end,end","bitter,bitter,flavour,flavour","grams,grams,grams,grams","Fresh crab with lemon,Garlic butter dipping sa...",18
3,Berliner Weisse With Yuzu - B-Sides,8.0,IBU Low,1,4.0,Pale Gold,3,4.2,ABV Low,1,...,Clean everything twice. All you want is the cl...,IBU Low Pale Gold ABV Low Atten High,"Propino Pale Malt,Wheat Malt,Propino Pale Malt...","kilograms,kilograms,kilograms,kilograms",Bramling Cross,middle,bitter,grams,"Smoked chicken wings,Miso ramen,Yuzu cheesecake",50
4,Pilsen Lager,55.0,IBU High,3,15.0,Deep Amber,8,6.3,ABV Low,1,...,Play around with the fermentation temperature ...,IBU High Deep Amber ABV Low Atten Medium,"Extra Pale,Caramalt,Dark Crystal,Munich","kilograms,kilograms,kilograms,kilograms","Centennial,Amarillo,Centennial,Amarillo,Centen...","start,start,middle,middle,end,end","bitter,bitter,flavour,flavour,flavour,flavour","grams,grams,grams,grams,grams,grams","Spicy crab cakes,Spicy cucumber and carrot Tha...",18
5,Avery Brown Dredge,59.0,IBU High,3,5.0,Pale Gold,3,7.2,ABV Low,1,...,Make sure you have a big enough yeast starter ...,IBU High Pale Gold ABV Low Atten Low,"Lager Malt,Wheat","kilograms,kilograms","Saaz,Saaz,Saaz","start,middle,end","bitter,flavour,flavour","grams,grams,grams","Vietnamese squid salad,Chargrilled corn on the...",35


In [107]:
output_df.to_csv('output_df.csv')


In [108]:
output_df.dtypes

name                         object
ibu                         float64
ibu_category                 object
ibu_category_nbr              int64
srm                         float64
srm_category                 object
srm_category_nbr              int64
abv                         float64
abv_category                 object
abv_category_nbr              int64
ebc                         float64
attenuation_level           float64
attenuation_category         object
attenuation_category_nbr      int64
description                  object
image_url                    object
ph                          float64
tagline                      object
brewers_tips                 object
outcome                      object
malts_name                   object
amount_unit_x                object
hops_name                    object
hops_add                     object
hops_attribute               object
amount_unit_y                object
food_pairing                 object
ai_prediction               

In [109]:
connection_string = "db/beerdata.sqlite"
engine = create_engine(f'sqlite:///{connection_string}')
engine.execute('''CREATE TABLE finaldata (
    id INT,
    name TEXT,
    ibu FLOAT,
    ibu_category TEXT,
    ibu_category_nbr FLOAT,
    srm FLOAT,
    srm_category TEXT,
    srm_category_nbr FLOAT,
    abv FLOAT,
    abv_category TEXT,
    abv_category_nbr FLOAT,
    ebc FLOAT,
    attenuation_level FLOAT,
    attenuation_category TEXT,
    attenuation_category_nbr FLOAT,
    description TEXT,
    image_url TEXT,
    ph FLOAT,
    tagline TEXT,
    brewers_tips TEXT,
    outcome TEXT,
    malts_name TEXT,
    amount_unit_x TEXT,
    hops_name TEXT,
    hops_add TEXT,
    hops_attribute TEXT,
    amount_unit_y TEXT,
    food_pairing TEXT,
    ai_prediction FLOAT,
    primary key("id"));''')

output_df.to_sql(name='finaldata', con=engine, index=False, if_exists='append')